## Markowitz方差组合模型

## 目录
1. 什么是Markowitz方差组合模型？
2. Portfolioopt的输入格式是什么？
3. 如何计算组合风险？
4. 如何将Markowitz可视化？

## 什么是Markowitz方差组合模型？
马克维茨投资组合理论的基本假设为：

(1)投资者是风险规避的，追求期望效用最大化；

(2)投资者根据收益率的期望值与方差来选择投资组合；

(3)所有投资者处于同一单期投资期。马克维茨提出了以期望收益及其方差(E，δ2)确定有效投资组合。

In [ ]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import portfolioopt as opt

symbol=['600036.XSHG','600050.XSHG','601318.XSHG']
PN = pd.Panel(pd.read_excel('sz50.xlsx', sheet_name = ['600036.XSHG','600050.XSHG','601318.XSHG'], index_col='datetime'))
PN

In [ ]:
PN.pct_change(axis=1).xs('close', axis=2)

## Portfolioopt的输入格式是什么？
安装Portfolioopt: pip install Portfolioopt

In [ ]:
PN.pct_change(axis=1).xs('close', axis=2)

In [ ]:
data_r = PN.minor_xs('close').pct_change()[1:]
print(data_r)

## Q: 关于矩阵的形状？

In [ ]:
#主要的两个输入模式
exp_rets = data_r.mean()*252
cov_mat = data_r.cov()*252
print(exp_rets)
print(cov_mat)

In [ ]:
pd.get_option('precision')

In [ ]:
#计算目标收益的权重
portfolio_1 = opt.markowitz_portfolio(cov_mat, exp_rets, 0.5, allow_short=False, market_neutral=False)
print(portfolio_1)

In [ ]:
#计算目标收益的权重
portfolio_1 = opt.markowitz_portfolio(cov_mat, exp_rets, 0.5, allow_short=False, market_neutral=False)
print(portfolio_1)

In [ ]:
#计算目标收益的权重
omega1 = opt.markowitz_portfolio(cov_mat, exp_rets, 0.59, allow_short=False, market_neutral=False) 
omega2 = opt.markowitz_portfolio(cov_mat, exp_rets, 0.6, allow_short=False, market_neutral=False)
omega1 == omega2

In [ ]:
#计算目标收益的权重
pd.set_option('precision', 10)
omega1 = opt.markowitz_portfolio(cov_mat, exp_rets, 0.59, allow_short=False, market_neutral=False) 
omega2 = opt.markowitz_portfolio(cov_mat, exp_rets, 0.6, allow_short=False, market_neutral=False)
print(omega1, omega2)
pd.reset_option('precision')

In [ ]:
#计算最小方差的权重
portfolio_mv = opt.min_var_portfolio(cov_mat, allow_short=False)
print(portfolio_mv)

In [ ]:
#计算最优组合的权重
portfolio_tp = opt.tangency_portfolio(cov_mat, exp_rets, allow_short=False)
print(portfolio_tp)

In [ ]:
#去除少于0.01权重的股票
weigth_t = opt.truncate_weights(portfolio_tp, min_weight=0.01, rescale=True)
print(weigth_t)

## 如何计算组合风险？
$Portfolio Variance = x'\sum x$

x = weights

$\sum = Cov$

In [ ]:
Portfolio_v = np.dot(weigth_t.T, np.dot(cov_mat,weigth_t))
P_sigma = np.sqrt(Portfolio_v)
print(P_sigma)

## 如何将Markowitz可视化？

In [ ]:
port_returns = []
port_variance = []
for p in range(4000):
    weights = np.random.rand(len(data_r.T))
    weights /= np.sum(weights)
    port_returns.append(np.sum(data_r.mean()*252*weights))
    port_variance.append(np.sqrt(np.dot(weights.T, np.dot(cov_mat,weights))))
port_returns = np.array(port_returns)
port_variance = np.array(port_variance)

risk_free = 0.04
P_r = np.sum(exp_rets*weigth_t)
sharpe = (P_r-risk_free)/P_sigma
print('sharpe:', sharpe)

plt.figure(figsize = (15,7))
plt.scatter(port_variance, port_returns, c=(port_returns-risk_free)/port_variance, marker='o')
plt.grid(True)
plt.xlabel('excepted volatility')
plt.ylabel('expected return')
plt.colorbar(label='Sharpe ratio')
plt.scatter(P_sigma, P_r, c='r', marker='*')
plt.show()

## 作业
选择三只或以上的股票，用Portfolioopt 将最优组合算出来，并将其可视化。